In [3]:
from weaviate.collections.classes.config import VectorDistances
%pip install -U weaviate-client
%pip install -U bitarray

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/pty.py:95: DeprecationWarning: This process (pid=88810) is multi-threaded, use of forkpty() may lead to deadlocks in the child.
  pid, fd = os.forkpty()


Note: you may need to restart the kernel to use updated packages.
  Obtaining dependency information for bitarray from https://files.pythonhosted.org/packages/80/9f/4dd05086308bfcc84ad88c663460a8ad9f5f638f9f96eb5fa08381054db6/bitarray-3.0.0-cp312-cp312-macosx_11_0_arm64.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.2/121.2 kB 6.3 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [13]:
import weaviate
from weaviate.classes.config import Property, Configure, DataType, VectorDistances

client : weaviate.WeaviateClient = weaviate.connect_to_local()
collectionName = "BinaryIndex"


In [14]:
if client.collections.exists(collectionName):
    client.collections.delete(collectionName)

client.collections.create(
    name=collectionName,
    properties=[
        Property(name="imageTitle", data_type=DataType.TEXT),
        Property(name="perceptualHashes", data_type=DataType.BLOB)
    ],
    vector_index_config=Configure.VectorIndex.hnsw(
        distance_metric=VectorDistances.HAMMING,
        quantizer=Configure.VectorIndex.Quantizer.bq(rescore_limit=0)
    ),
    vectorizer_config=Configure.Vectorizer.none()
)

In [15]:
import bitarray

def hash_to_vecotor(hash: str) -> [float]:
    ba = bitarray.bitarray()
    ba.frombytes(hash.encode('utf-8'))
    vector = []
    for bit in ba:
        vector.append(-1 if bit <= 0 else 1)

    return vector

In [16]:
hashes = ["ffd7918181c9ffff", "ffd7918181c9afff", "9f172786e71f1e00", "9f172786e72f1e00"]

with client.batch.dynamic() as batch:
    for hash in hashes:
        batch.add_object(
            collection=collectionName,
            properties={
                "imageTitle": hash,
                "perceptualHashes": hash,
            },
            vector=hash_to_vecotor(hash)
        )

In [18]:
from weaviate.collections.classes.grpc import MetadataQuery

collection = client.collections.get(collectionName)
response = collection.query.near_vector(
    near_vector=hash_to_vecotor("ffd7918181c9afff"),
    limit=2,
    return_metadata=MetadataQuery(distance=True)
)

for o in response.objects:
    print(o.properties)
    print(o.metadata.distance)

{'imageTitle': 'ffd7918181c9afff'}
0.0
{'imageTitle': 'ffd7918181c9ffff'}
3.0
